# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
print(salaries.shape)
salaries.head()

(33183, 8)


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:

#we use the mean instead of sum to check the percentage of missing values
salaries.isnull().mean()

Name                 0.00000
Job Titles           0.00000
Department           0.00000
Full or Part-Time    0.00000
Salary or Hourly     0.00000
Typical Hours        0.75825
Annual Salary        0.24175
Hourly Rate          0.75825
dtype: float64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:

salaries.Name.groupby(salaries["Salary or Hourly"]).count()

Salary or Hourly
Hourly     8022
Salary    25161
Name: Name, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
salaries.Name.groupby(salaries["Department"]).count()

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# Your code here:

#they are asking us to test the mean of our sample vs the mean of the population with a 95% confidence

#To do so we will use 1 sample t test or z test
#In this case we will use t test as we don't know the population variance

#Also the resulted p-value should divided by 2, as it is a 2 sided test, however scipy 1 sample t test 
#compute the 2 tailed p-value, so it is automatically divided by 2 and therefore we don't have to do p-value/2

#we assume h0 as sample mean = $30/hr
hourlyWage = salaries["Hourly Rate"][salaries["Hourly Rate"].notna()]

ttest_1samp = stats.ttest_1samp(hourlyWage, 30.0)

print(ttest_1samp,"\n")

if ttest_1samp[1] < 0.05:
    print("With a 5% confidence level, we can reject h0 and assume hourly wage is significantly different from $30/hr")
else:
    print("With a 5 % confidence level we cannot reject h0")

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92) 

With a 5% confidence level, we can reject h0 and assume hourly wage is significantly different from $30/hr


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [16]:
# Your code here:

#Again we have to test a sample mean vs pop mean without knowing pop variance, so we will use t test
#However now is 1 tailed as we'd like to know if the sample mean is bigger than the pop mean
#As ttest_1samp give us the 2 tailed p-value, that is p-value/2, now we will have to p-value*2

#Our h0: this year police salaries mean <= previous year mean

policeSalary = salaries["Annual Salary"][(salaries["Department"]=="POLICE") & (salaries["Annual Salary"].notna())]

ttest_1samp1 = stats.ttest_1samp(policeSalary, 86000.0)

print(ttest_1samp1[0],ttest_1samp1[1]*2,"\n")

if ttest_1samp1[1]*2 < 0.05:
    print("With a 5% confidence level, we can reject h0 and assume police salaries mean is higher than 86k$/year")
else:
    print("With a 5 % confidence level we cannot reject h0")

3.081997005712994 0.0041206807101930275 

With a 5% confidence level, we can reject h0 and assume police salaries mean is higher than 86k$/year


Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
# Your code here:
mostHr_workers = pd.crosstab(salaries["Department"],salaries["Salary or Hourly"][salaries["Salary or Hourly"]=="Hourly"]).sort_values("Hourly", ascending=False).head(1)
mostHr_workers

Salary or Hourly,Hourly
Department,
STREETS & SAN,1862


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [10]:
# Your code here:

#Now the test follows the same rules as the previous one, so we will use again 1 tailed t test
#Althought now the test will check if the houlry wage mean is less than 35$
#However, besides h0, the test doesn't change, just the conclusion

#h0: street and sanitary hourly salary >= 35$/hour

street_sanSalary = salaries["Hourly Rate"][(salaries["Department"]=="STREETS & SAN") & (salaries["Hourly Rate"].notna())]

ttest_1samp2 = stats.ttest_1samp(street_sanSalary, 35.0)

print(ttest_1samp2[0],ttest_1samp2[1]*2,"\n")

if ttest_1samp2[1]*2 < 0.05:
    print("With a 5% confidence level, we can reject h0 and assume street and sanitary hourly salary range mean is less than 35$/hour")
else:
    print("With a 5 % confidence level we cannot reject h0")

-9.567447887848152 6.6757061129415434e-21 

With a 5% confidence level, we can reject h0 and assume street and sanitary hourly salary range mean is less than 35$/hour


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [11]:
# Your code here:

#Here we reafirm our previous test that hourlyWage is diff from 30

CI = stats.t.interval(0.95, len(hourlyWage)-1, hourlyWage.mean(), stats.sem(hourlyWage))
CI

(32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [12]:
# Your code here:

#Here we reafirm our previous test that policeSalary is higher than 86k

CI1 = stats.t.interval(0.95, len(policeSalary)-1, policeSalary.mean(), stats.sem(policeSalary))
CI1

(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [15]:
# Your code here:

#h0: hourly workers in the City of Chicago = 25% of total workers

from statsmodels.stats import proportion

hWorkers = salaries.Name.groupby(salaries["Salary or Hourly"][salaries["Salary or Hourly"]=="Hourly"]).count()
observations = len(salaries)

z, pvalue = proportion.proportions_ztest(hWorkers, observations, value=0.25, alternative='two-sided')

print(z,pvalue,"\n")

if pvalue < 0.05:
    print("With a 5% confidence level, we can reject h0 and assume hourly workers in the City of Chicago is significantly different from 25%")
else:
    print("With a 5 % confidence level we cannot reject h0")

[-3.50999642] [0.00044811] 

With a 5% confidence level, we can reject h0 and assume hourly workers in the City of Chicago is significantly different from 25%


In [14]:
#he hecho el calculo para comprobar el test anterior y pepino lo acertado que es
#ya que si cambio a value=0.24  no rechaza h0

s = ((salaries.Name.groupby(salaries["Salary or Hourly"][salaries["Salary or Hourly"]=="Hourly"]).count())*100)/len(salaries)
s

Salary or Hourly
Hourly    24.175029
Name: Name, dtype: float64